In [3]:
import pandas as pd
import numpy as np


In [4]:
# решение с сайта
# считываем данные
qid_query = pd.read_csv('hostid_url.tsv', sep='\t', names=['qid', 'query'])
qid_url_rating = pd.read_csv('qid_url_rating.tsv', sep='\t', names=['qid', 'url', 'rating'])
hostid_url = pd.read_csv('hostid_url.tsv', sep='\t', names=['hostid', 'url'])

# делаем join двух таблиц, чтобы было просто брать url с максимальным рейтингом
qid_url_rating_hostid = pd.merge(qid_url_rating, hostid_url, on="url")


def plook(ind, rels):
    if ind == 0:
        return 1
    return plook(ind-1, rels)*(1-rels[ind-1])*(1-0.15)

def pfound(group):
    max_by_host = group.groupby("hostid")["rating"].max() # максимальный рейтинг хоста
    top10 = max_by_host.sort_values(ascending=False)[:10] # берем топ-10 урлов с наивысшим рейтингом
    pfound = 0
    for ind, val in enumerate(top10):
        pfound += val*plook(ind, top10.values)
    return pfound

In [5]:
qid_pfound = qid_url_rating_hostid.groupby('qid').apply(pfound) # группируем по qid и вычисляем pfound
qid_max = qid_pfound.idxmax() # берем qid с максимальным pfound

qid_query[qid_query["qid"] == qid_max]

,qid,query


In [6]:
def PFound(qid_url_rating_hostid, K=10, p_break=0.15):

    # Ищем наибольший рейтинг для кажого url
    max_by_host = qid_url_rating_hostid.groupby(['qid', 'hostid'])['rating'].max().reset_index()
    top = max_by_host.sort_values(['qid', 'rating'], ascending=False).groupby(['qid']).head(K)

    # задаем порядок для отсортированного списка
    top['rank'] = top.groupby('qid').cumcount() 
    
    # считаем pfound 
    top['omr'] = (1 - top['rating']).shift(1)
    top['ompbr'] = 1 - p_break
    top.loc[top['rank'] == 0, ['ompbr', 'omr']] = 1

    top['omr_ompbr'] = top['omr'] * top['ompbr']

    top['plook'] = top.groupby('qid')['omr_ompbr'].cumprod()
    top['pfound'] = top['plook'] * top['rating']

    pfound_qid = top.groupby('qid')['pfound'].sum() 
    return pfound_qid.mean()



0.4603173929969002


In [ ]:
print(PFound(qid_url_rating_hostid))